In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,10929
2,Huelva,Confirmados PDIA 14 días,900
3,Huelva,Tasa PDIA 14 días,"176,21386881464846"
4,Huelva,Confirmados PDIA 7 días,457
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,1
629,Municipio de Huelva sin especificar,Total Confirmados,123
630,Municipio de Huelva sin especificar,Curados,102


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  10929.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3091.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1049 personas en los últimos 7 días 

Un positivo PCR cada 515 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,10929.0,457.0,900.0,510743.0,89.477487,176.213869,75.0
Huelva-Costa,6173.0,321.0,615.0,288115.0,111.413845,213.456432,58.0
Isla Cristina,436.0,45.0,94.0,21264.0,211.625282,442.061701,18.0
Condado-Campiña,3350.0,117.0,253.0,155057.0,75.456123,163.165804,17.0
Huelva (capital),3091.0,137.0,279.0,143663.0,95.362063,194.204492,16.0
Lepe,961.0,45.0,77.0,27431.0,164.047975,280.704313,10.0
Cartaya,400.0,42.0,71.0,19974.0,210.273355,355.462101,6.0
Palma del Condado (La),528.0,23.0,72.0,10761.0,213.734783,669.082799,5.0
Bonares,56.0,11.0,14.0,6058.0,181.578079,231.099373,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Palma del Condado (La),528.0,23.0,72.0,10761.0,213.734783,669.082799,5.0
Lucena del Puerto,99.0,14.0,17.0,3371.0,415.307031,504.301394,1.0
Jabugo,46.0,10.0,11.0,2250.0,444.444444,488.888889,0.0
Isla Cristina,436.0,45.0,94.0,21264.0,211.625282,442.061701,18.0
Cartaya,400.0,42.0,71.0,19974.0,210.273355,355.462101,6.0
Palos de la Frontera,239.0,15.0,33.0,11289.0,132.872708,292.319957,1.0
Lepe,961.0,45.0,77.0,27431.0,164.047975,280.704313,10.0
Aljaraque,432.0,30.0,56.0,21260.0,141.110066,263.405456,3.0
Escacena del Campo,97.0,6.0,6.0,2284.0,262.697023,262.697023,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Bollullos Par del Condado,249.0,3.0,18.0,14272.0,21.020179,126.121076,0.0,0.166667
San Bartolomé de la Torre,45.0,1.0,4.0,3736.0,26.766595,107.066381,0.0,0.250000
Palma del Condado (La),528.0,23.0,72.0,10761.0,213.734783,669.082799,5.0,0.319444
Minas de Riotinto,67.0,1.0,3.0,3848.0,25.987526,77.962578,0.0,0.333333
Aroche,55.0,1.0,3.0,3073.0,32.541490,97.624471,0.0,0.333333
Almonte,439.0,2.0,6.0,24191.0,8.267538,24.802613,0.0,0.333333
San Juan del Puerto,153.0,6.0,16.0,9300.0,64.516129,172.043011,0.0,0.375000
Punta Umbría,234.0,4.0,9.0,15242.0,26.243275,59.047369,1.0,0.444444
Palos de la Frontera,239.0,15.0,33.0,11289.0,132.872708,292.319957,1.0,0.454545
